这是一个关于从搜索引擎上下载图片的教程。
使用的环境是：python 3。
用到的库有beautiful soup4和requests。
使用的搜索引擎是雅虎日本的图片搜索引擎。我也尝试过爬取谷歌的图片搜索，我只找到了缩略图的路径。所以这个notebook里面就用雅虎日本为例子进行说明。

In [1]:
#导入相关的库文件
import requests
import shutil
import os
from bs4 import BeautifulSoup

其实爬虫的原理挺简单的。拿到一个网站的地址后，按图索骥，找到对应的页面并且下载。然后在这个页面文件中找到图片的地址就可以了。我们先构建一下这个地址。

In [2]:
#松屋是一家日本的牛肉饭快餐提供商，我准备从搜索引擎中依据该关键词下载图片
query = "松屋"
#url的构建需要多尝试几下，然后找到 query word应该被放在哪里
url = 'https://search.yahoo.co.jp/image/search;_ylt=A2RivbCKbvVZYhQA9SKU3uV7?p=' + query + '&aq=-1&oq=&ei=UTF-8'
#很多网站都会禁止机器人访问，所以在这里加上header伪装成人
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
#requests比较好用，比较少有格式之间的问题
r = requests.get(url, headers = headers)
#用 r.text 获取文本的信息，选用的解析器是 html.parser，也可以选择别的解析器
soup = BeautifulSoup(r.text, 'html.parser')


Beautiful Soup这个工具可以帮我们构建一棵树，让我们可以快速找到我们需要的信息所处的位置。我们用chrome浏览器查看网页的结构（inspect）。
![title](snip1.PNG)
我在这里截取了一部分，要找的图片路径就是有a标签，href，target属性（attribute）的那一个格子里面。知道这些了，用Beautiful Soup可以快速的获得。

In [4]:
#定义一个过滤器。bs4还有很多用法，需要的时候可以进行查询
def has_class_but_no_id(tag):
    return tag.has_attr('href') and tag.has_attr('target')
ll = soup.find_all(has_class_but_no_id)
#这里存着图片对应的地址
image_urls= []
for i in ll:
    c = str(i)
    d = c.split(" ")
    print("-------------------------------------")
    print (d[1][6:-1])
    image_urls.append(d[1][6:-1])

-------------------------------------
https://ord.yahoo.co.jp/o/image/RV=1/RE=1509358279/RH=b3JkLnlhaG9vLmNvLmpw/RB=/RU=aHR0cDovL21jaGEtdHcuY29tL3dwLWNvbnRlbnQvdXBsb2Fkcy8yMDE0LzA1LyVFNyU4OSU5QiVFNCVCOCVCQzQuanBn/RS=^ADBxt9Jml44q4rKBnbfwV8s2ukn9BY-;_ylt=A2RivchGqfVZz0cA6TuU3uV7
-------------------------------------
https://ord.yahoo.co.jp/o/image/RV=1/RE=1509358279/RH=b3JkLnlhaG9vLmNvLmpw/RB=/RU=aHR0cDovL21jaGEtdHcuY29tL3dwLWNvbnRlbnQvdXBsb2Fkcy8yMDE0LzA1LyVFNyU4OSU5QiVFNCVCOCVCQzQuanBn/RS=^ADBxt9Jml44q4rKBnbfwV8s2ukn9BY-;_ylt=A2RivchGqfVZz0cA6DuU3uV7
-------------------------------------
https://ord.yahoo.co.jp/o/image/RV=1/RE=1509358279/RH=b3JkLnlhaG9vLmNvLmpw/RB=/RU=aHR0cHM6Ly90YmxnLmstaW1nLmNvbS9yZXNpemUvNjYweDM3MGMvcmVzdGF1cmFudC9pbWFnZXMvUnZ3LzU0NDU2LzU0NDU2NjU2LmpwZz90b2tlbj04MTQ5ZTRkJmFwaT12Mg--/RS=^ADBxT2Daj8OAA9nyV0agxEf6ZacAf8-;_ylt=A2RivchGqfVZz0cA6zuU3uV7
-------------------------------------
https://ord.yahoo.co.jp/o/image/RV=1/RE=1509358279/RH=b3JkLnlhaG9vLmNvLmpw/RB=/RU

拿到图片的地址后距离成功就相当的进啦。创建一个文件夹（我用了和查询词相同的名字）用于保存这些图片，然后下载就好了。注意到拿到的图片地址并不一定全部有效，所以需要一个判别的过程。其实很简单，直接看代码好了。

In [5]:
if not (os.path.exists(query)):
    os.mkdir(query)
    
#其实还是有点不完善，image_urls里的最后一个项目其实不是图片地址    
for index, url in enumerate(image_urls[:-1]):
    try:
        r = requests.get(url, timeout = 3, headers = headers, stream = True)
        with open(query + "/" + query + str(index) + '.png', 'wb') as out_file:
            shutil.copyfileobj(r.raw, out_file)
        del r
    except requests.exceptions.ConnectionError:
        print("can't open this page")


can't open this page
can't open this page


可以看到有两个连接是打不开的，用try...catch后就避免了exception扔出来后就死掉了的悲剧。爬下来的图片像这样：
![title](snip2.PNG)

小结一下：

一. 关于Jupyter Notebook使用的小技巧：1.怎么插入文字。依次按下Esc，m，Enter就可以输入文字，点击一下执行按钮就可以退出写文字的部分。
2.怎么插入图片。![title](picture_name.PNG)，注意不要用引号哦。

二. 这里只是介绍了一个爬虫的小例子。预定以后还需要学习怎么用正则进行更复杂的匹配（正则入门了好多次，都没有在这种场合真实用过），多了解一些requests的用法。
